In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import os
import re
import math
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime
from os.path import join
import random
import requests

In [ ]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.18.1 requires transformers>=4.50.0, but you have transformers 4.48.3 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"
from google.colab import userdata

hf_token = userdata.get('aaaa')
!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="cuda",  # "auto" يعمل بشكل جيد أيضًا
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

Memory footprint: 16.1 GB


In [ ]:
finetuned_model_id = "ahmed-ouka/llama-lora-adapter-eniad"
base_model.load_adapter(finetuned_model_id)

adapter_config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [1]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

In [2]:
intent_category = Literal[
    "acadimique information", "Admissions et inscriptions", "Services aux étudiants",
    "Droits et responsabilités des étudiants", "Activités étudiantes", "Services administratifs",
    "Vie sur le campus", "Autre"
]
class RelatedQuestion(BaseModel):
    question1: str = Field(description="Question suggérée liée à la réponse et à la question posée" )
    question2: str = Field(description="Question suggérée liée à la réponse et à la question posée" )
    question3: str = Field(description="Question suggérée liée à la réponse et à la question posée" )

class ChatBotResponse(BaseModel):
    contentment: str = Field(
    ...,
    min_length=20,
    max_length=80,
    description=(
        "Exprimez poliment que vous avez bien compris la question. "
        "Utilisez une phrase courte, rassurante, naturelle et adaptée au contexte de la question. "
        "Par exemple : 'Merci pour votre question', 'Je comprends votre demande', 'سؤال جيد، شكرًا لك'. "
        "Répondez dans la même langue que celle de la question."
                 )
         )
    main_answer: str = Field(
    ...,
    min_length=20,
    max_length=100,
    description=(
        "Fournissez une réponse directe, claire et concise à la question posée. "
        "Évitez les longueurs inutiles. "
        "Répondez dans la même langue que celle utilisée dans la question."
                 )
           )
    details: Optional[str] = Field(
    None,
    min_length=20,
    max_length=500,
    description=(
        "Ajoutez des explications ou informations supplémentaires pour enrichir la réponse si nécessaire. "
        "Incluez des exemples, contextes ou précisions utiles. "
        "Répondez dans la même langue que celle de la question."
                 )
           )

    intent: intent_category = Field(
    ...,
    description=(
        "Identifiez l'intention principale de la question."
                )
           )

    related_questions: Optional[List[RelatedQuestion]] = Field(
    None,
    description=(
        "Une liste de questions similaires ou couramment posées en lien avec la question actuelle. "
        "Répondez dans la même langue que celle de la question d'origine."
                 )
             )

In [3]:
import json

In [12]:
prompte_Task_messages = [
     {
        "role": "system",
        "content": "\n".join([
            "Vous êtes un assistant dans une école qui s'appelle l'École Nationale de l'Intelligence Artificielle et Digitale de Berkane, capable de répondre aux questions des étudiants.",
            "Vérifiez les données attentivement lorsque vous répondez.",
            "Essayez d'éviter tous les mots et textes qui n'ont pas de sens dans ces données.",
            "Faites très attention à la langue dans laquelle la question est posée.",
            "Vous devez répondre dans la même langue que celle de la question.",
            "Ne répondez pas tant que vous n’êtes pas sûr de la langue de la question.",
            "Si c'est en arabe, répondez en arabe. Si c'est en français, répondez en français. Si c'est en anglais, répondez en anglais.",
            "Ignorez les éléments inutiles dans la question tels que les numéros de version ou de commande, et concentrez-vous uniquement sur la question.",
            "Faites attention aux fautes d'orthographe pour ne pas altérer votre compréhension.",
            # "Extraire les détails JSON du texte conformément aux questions posées et aux spécifications Pydantic.",
            "Extraire les détails comme indiqué dans le texte. Vous pouvez les reformater, mais gardez le sens.",
            "Ne pas générer d'introduction ni de conclusion.",
            "repandre en paragraphe text"
            "n'oblier pas les questions similaires a la fin au moins deux mais Ils doivent être présentés avec le contexte du texte et ne doivent en aucun cas être mentionnés auparavant. Indiquez simplement à l'utilisateur que vous pouvez l'aider avec d'autres choses, puis posez des questions. "
            "",
            "Ne vous limitez pas toujours à la première phrase de votre question, très bien, merci, mais diversifiez plutôt la phrase.",


        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Question : عرف لي هده المدرسة ",



            "## Pydantic Details:",
            json.dumps(
                ChatBotResponse.model_json_schema(), ensure_ascii=False
            ),
            "",
            "## Output text:",


        ])
    }
]

In [ ]:
text = tokenizer.apply_chat_template(
    prompte_Task_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

generated_ids = base_model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    temperature=0.3, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(response)

سؤال جيد، شكرًا لك.

المدرسة الوطنية للدكاء الاصطناعي هي إحدى المؤسسات التعليمية التي تهدف إلى تعزيز تطوير مهارات الطلاب في مجال الذكاء الاصطناعي والتقنيات الرقمية.

تفاصيل إضافية: تأسست المدرسة في Berkane، وتهدف إلى تلبية احتياجات السوق الرقمي من خلال تدريب مهندسين قادرين على تلبية احتياجات الصناعة.

يمكن للطلاب متابعة برامج دراسية في مجالات مثل الذكاء الاصطناعي، وتطوير البرمجيات، والشبكات، والائتمانية، والتعلم الآلي، والروبوتات، والواقع الافتراضي، والذكاء الاصطناعي في الأمن السيبراني.

المدرسة الوطنية للدكاء الاصطناعي تعمل في شراكة مع الصناعة المحلية و国际ًا لضمان أن برامجها تتضمن المعايير العالمية وتلبي احتياجات السوق.

يمكن للطلاب أيضًا المشاركة في مشاريع عملية، والتعاون مع الشركات، والتدريب العملي في مجالات متقدمة من الذكاء الاصطناعي.

المدرسة الوطنية للدكاء الاصطناعي تهدف إلى أن تكون مركزًا للتميز في الذكاء الاصطناعي والتقنيات الرقمية في المغرب، وتسعى إلى تطوير مهندسين قادرين على الابتكار والتفوق.

يمكنك الاستفسار حول كيفية التسجيل في المدرسة أو الحصول على معلومات إضافية عن برامجها

## vLLM

In [7]:
!pip install -qU vllm==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/2

In [1]:
base_model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
adapter_model_id = "ahmed-ouka/llama-lora-adapter-eniad"


!nohup vllm serve "{base_model_id}" --dtype=half  --max-model-len 8192  --gpu-memory-utilization 0.8 --max_lora_rank 64 --enable-lora --lora-modules news-lora="{adapter_model_id}"&

nohup: appending output to 'nohup.out'


In [13]:
!tail -n 30 nohup.out

INFO 06-08 22:14:32 model_runner.py:1562] Graph capturing finished in 31 secs, took 0.40 GiB
INFO 06-08 22:14:32 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 36.75 seconds
INFO 06-08 22:14:32 api_server.py:756] Using supplied chat template:
INFO 06-08 22:14:32 api_server.py:756] None
INFO 06-08 22:14:33 serving_models.py:174] Loaded new LoRA adapter: name 'news-lora', path 'ahmed-ouka/llama-lora-adapter-eniad'
INFO 06-08 22:14:33 launcher.py:21] Available routes are:
INFO 06-08 22:14:33 launcher.py:29] Route: /openapi.json, Methods: GET, HEAD
INFO 06-08 22:14:33 launcher.py:29] Route: /docs, Methods: GET, HEAD
INFO 06-08 22:14:33 launcher.py:29] Route: /docs/oauth2-redirect, Methods: GET, HEAD
INFO 06-08 22:14:33 launcher.py:29] Route: /redoc, Methods: GET, HEAD
INFO 06-08 22:14:33 launcher.py:29] Route: /health, Methods: GET
INFO 06-08 22:14:33 launcher.py:29] Route: /ping, Methods: GET, POST
INFO 06-08 22:14:33 launcher.py:29] Route: /tokenize, Methods

In [14]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

prompt = tokenizer.apply_chat_template(
    prompte_Task_messages,
    tokenize=False,
    add_generation_prompt=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [24]:
vllm_model_id = "news-lora"

llm_response = requests.post("http://localhost:8000/v1/completions", json={
    "model": vllm_model_id,
    "prompt": prompt,
    "max_tokens": 1024,
    "temperature": 0.3
})
response_data = llm_response.json()
generated_text = response_data['choices'][0]['text']
print(generated_text)

سؤال جيد، شكرًا لك.

المدرسة الوطنية للدكاء الاصطناعي هي المدرسة التي تهدف إلى تعزيز التعليم والتدريب في مجالات الذكاء الاصطناعي والتقنيات الرقمية في بركان، المغرب.

تفاصيل إضافية: تأسست المدرسة في [تاريخ التأسيس] وتتركز على تقديم برامج دراسية تشمل شهادات ودرجات علمية في الذكاء الاصطناعي والتقنيات المتعلقة. تهدف إلى تلبية احتياجات السوق الرقمي وتوفير مهندسين قادرين على الابتكار والتفوق.

يمكن للطلاب التسجيل في برامج الدكتوراه والماجستير والlicence في مجالات مثل الذكاء الاصطناعي، الشبكات، والأمن السيبراني، وغيرها.

المدرسة الوطنية للدكاء الاصطناعي تعمل في شراكة مع الصناعة المحلية والدولية لضمان أن برامجها تتوافق مع احتياجات السوق وتتيح للطلاب فرص التدريب والتوظيف.

يمكن للطلاب أيضًا الاستفادة من مرافق المدرسة الحديثة، بما في ذلك مكتبة رقمية، مركز للتدريب، وفرص للبحث والابتكار.

المدرسة الوطنية للدكاء الاصطناعي تهدف إلى becoming مركزًا للتميز في الذكاء الاصطناعي والتقنيات الرقمية في المنطقة، وتساهم في تطوير الاقتصاد المحلي وتحفيز الابتكار.

أماكن مشابهة يمكن أن تثير الفضول:

* ما هي البر

In [6]:
!pip install openai

In [20]:

import openai

client = openai.OpenAI(
    base_url="https://gorafrenk--llama3-openai-compatible-serve.modal.run/v1",
    api_key="super-secret-key",
)

response = client.chat.completions.create(
    model="ahmed-ouka/llama3-8b-eniad-merged-32bit",
    messages=prompte_Task_messages,
    temperature=0.2,
    max_tokens=1024
)

print(response.choices[0].message.content)

سؤال جيد، شكرًا لك.

المدرسة التي تدرس فيها هي المدرسة الوطنية للذكاء الاصطناعي والتقنية الرقمية في بركان.

تأسست هذه المدرسة بهدف تكوين مهندسين في مجالات الذكاء الاصطناعي والتقنية الرقمية، مما يساهم في تطوير الاقتصاد الوطني.

المدرسة تقدم برامج دراسية متكاملة تشمل دورات في البرمجة، الشبكات، الأمن السيبراني، والروبوتات، بالإضافة إلى مشاريع عملية بالتعاون مع الصناعة.

الغواء: يمكنني مساعدتك مع أي سؤال آخر لديك؟ ما هي التخصصات المتاحة في المدرسة؟
